In [1]:
import os
import json

In [2]:
def flatten(l):
    fl = [item for sublist in l for item in sublist]
    return fl

In [ ]:
import numpy
l = numpy.array_split(remaining_titles,len(remaining_titles)/20);

In [3]:
import json

In [4]:
datasets_path = "/Users/sophierand/RCDatasets/datasets.json"
with open(datasets_path) as json_file:
    datasets = json.load(json_file)

In [5]:
sed_datasets = [{'id':d['id'],'search_terms':[d['title']] + d['alt_title']} for d in datasets if d['id'] == 'dataset-370']
sed_datasets

[{'id': 'dataset-370',
  'search_terms': ['Survey of Earned Doctorates', 'SED', 'SESTAT']}]

In [6]:
import dimensions_search_api_client as dscli
import RichContextAPI

In [7]:
api_client = RichContextAPI.connect_dimensions_api()

In [ ]:
# # ''
# search_string = 'search publications where doi in ["10.1002/9781119556091"] return publications[all]'
# api_response = api_client.execute_query(query_string_IN = search_string )

In [8]:
def run_exact_string_search(string,api_client):
    search_string = 'search publications in full_data for "\\"{}\\"" return publications[doi+title+journal+author_affiliations]'.format(string)
    api_response = api_client.execute_query(query_string_IN = search_string )
    return api_response

In [30]:
md_list = []
ds_id = sed_datasets[0]['id']
search_terms = sed_datasets[0]['search_terms']
author_values = []
for i in search_terms:
    dimension_return = run_exact_string_search(string = i,api_client = api_client)
    dimension_return_pub = dimension_return['publications']
    ds_pub_list = []
    for d in dimension_return_pub:
        try:
            md_dict = {'title':d['title'],'journal':d['journal']['title'],'search_string':i,'doi':d['doi']}
            if 'author_affiliations' in d.keys():
                author_list = d['author_affiliations'][0]
                for a in author_list:
                    auth_dict = {'first_name':a['first_name'],'last_name':a['last_name'],'affiliation':a['affiliations'][0]['name']}
                    author_values.append(auth_dict)
                md_dict.update({'authors':author_values})
            elif 'author_affiliations' not in d.keys():
                pass
            ds_pub_list.append(md_dict)
        except:
            pass
    md_list.append(ds_pub_list)

In [32]:
# md_dict

In [21]:
# d

In [33]:
md_list = flatten(md_list)

In [34]:
len(md_list)

57

In [37]:
# md_list

In [ ]:
for sub_list in l:
    md_list = []
    for s in sub_list:
        md_entry = grab_md(s)
        md_list.append(md_entry)
    file_name = get_filename()    
    with open(file_name, 'w') as outfile:
        json.dump(md_list, outfile, indent = 2)

In [ ]:
import dimensions_search_api_client as dscli
import RichContextAPI

In [ ]:
api_client = RichContextAPI.connect_dimensions_api()

In [ ]:
def run_exact_string_search(string,api_client):
    search_string = 'search publications in full_data for "\\"{}\\"" return publications'.format(string)
    api_response = api_client.execute_query(query_string_IN = search_string )
    return api_response

In [ ]:
a = run_exact_string_search(string = search_terms[1],api_client = api_client)

In [ ]:
dataset_returns = []
for d in ncses_datasets:
    ds_md = {'datasets':d['id']}
    search_terms = d['title']
    result_list = []
    for t in search_terms:
        ds_results = run_exact_string_search(string = t, api_client=api_client)
        md_results = ds_results['publications']
        [m.update(ds_md) for m in md_results]
        [m.update({'search_term':t}) for m in md_results]
        result_list.append(md_results)
#         result_list = flatten(result_list)
        dataset_returns.append(result_list)


In [ ]:
dimensions_results = flatten(flatten(dataset_returns))

In [ ]:
publications_md = []
final_df = pd.DataFrame()
for i,p in enumerate(dimensions_results):
    if isinstance(p,list):
        np = p[0]
    elif not isinstance(p,list):
        np = p
    p_dict = {'title':np['title'],'journal':np['journal']['title'],'dataset':np['search_term']}
    p_dict_df = p_dict
    try:
        author_list = np['author_affiliations'][0]
        author_values = []
        temp_df = pd.DataFrame()
        for a in author_list:
            the_name = a['first_name']+ ' '+a['last_name']+ ', '+a['affiliations'][0]['name']
            auth_dict = {'first_name':a['first_name'],'last_name':a['last_name'],'affiliation':a['affiliations'][0]['name']}
            author_values.append(the_name)
        auth_final = ",".join(author_values)
        p_dict_df.update({'authors':auth_final})
        p_df = pd.DataFrame(p_dict_df,index=[0])
        temp_df = temp_df.append(p_df, ignore_index=True)
            
#             p_dict.update({'authors':author_values})
#             publications_md.append(p_dict)
        final_df = final_df.append(temp_df,ignore_index = True)
    except:
        pass

In [ ]:
final_df = final_df.drop_duplicates()

In [ ]:
# final_df_export = final_df[['title','journal','first_name','last_name','affiliation','dataset']]
final_df.to_csv('ncses_example_linkages.csv')

In [ ]:
# final_df